In [82]:
%pip install numpy
%pip install scipy
%pip install matplotlib
%pip install neurokit2 
%pip install pandas
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\arche\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\arche\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\arche\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\arche\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\arche\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [83]:
import os
import pandas as pd
import scipy.io
import numpy as np
import neurokit2 as nk
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.signal import butter, filtfilt

def preprocess_signal(signal, original_fs=500, downsample_factor=10):
    """Preprocess the ECG signal by cleaning, downsampling, and filtering using a Butterworth filter."""
    try:
        # Clean the signal: Replace NaN, inf, and extreme values with 0 or appropriate value
        signal = np.nan_to_num(signal, nan=0.0, posinf=0.0, neginf=0.0)

        # Optionally remove any extreme values (e.g., values beyond a certain threshold)
        signal[signal > 10] = 0
        signal[signal < -10] = 0

        # Downsample the signal
        downsampled_signal = signal[::downsample_factor]

        # Now define the new sampling rate
        new_fs = original_fs / downsample_factor
        nyquist = 0.5 * new_fs

        # Define filter cutoffs relative to new sampling rate
        low_cutoff = 0.5   # Hz
        high_cutoff = 20.0 # Hz (keep a reasonable upper limit after downsampling)
        low = low_cutoff / nyquist
        high = high_cutoff / nyquist

        if high >= 1.0:
            high = 0.99  # safety margin if sampling too low

        # Design Butterworth bandpass filter
        b, a = butter(4, [low, high], btype='band')
        filtered_signal = filtfilt(b, a, downsampled_signal)

        return filtered_signal
    except Exception as e:
        print(f"Error preprocessing signal: {e}")
        return signal  # Return the original signal if preprocessing fails

        
def plot_ecg(signal, sampling_rate, title="ECG Signal"):
    """Plot the ECG signal."""
    try:
        time = np.arange(0, len(signal)) / sampling_rate
        plt.figure(figsize=(10, 4))
        plt.plot(time, signal)
        plt.title(title)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Amplitude")
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"Error plotting ECG signal: {e}")

def read_header_info(header_file):
    """Read metadata from the .hea header file and extract age, sex, and arrhythmia information."""
    try:
        with open(header_file, 'r') as f:
            lines = f.readlines()

        # Extract age, sex, and arrhythmia condition from the header
        age = sex = arrhythmia = None
        for line in lines:
            if line.startswith("#Age:"):
                age = float(line.split(":")[1].strip())
            elif line.startswith("#Sex:"):
                sex = line.split(":")[1].strip()
            elif line.startswith("#Dx:"):
                arrhythmia_codes = line.split(":")[1].strip().split(",")
                arrhythmia = any(code in arrhythmia_codes for code in [
                        "164889003",  # Atrial Fibrillation (AFIB)
                        "164890007",  # Atrial Flutter (AF)
                        "426761007",  # Supraventricular Tachycardia (SVT)
                        "713422000",  # Atrial Tachycardia (AT)
                        "233896004",  # Atrioventricular Node Reentrant Tachycardia (AVNRT)
                        "233897008",  # Atrioventricular Reentrant Tachycardia (AVRT)
                        "11157007",   # Ventricular Bigeminy (VB)
                        "75532003",   # Ventricular Escape Beat (VEB)
                        "17338001",   # Ventricular Premature Beat (VPB)
                        "195060002",  # Ventricular Preexcitation (VPE)
                        "251180001",  # Ventricular Escape Trigeminy (VET)
                        "74390002"    # Wolff-Parkinson-White Syndrome (WPW)
                    ])

        return {"Age": age, "Sex": sex, "Arrhythmia": arrhythmia}
    except Exception as e:
        print(f"Error reading header {header_file}: {e}")
        return None

def calculate_heart_features(signal, sampling_rate):
    """Calculate heart-related features such as BPM, RR intervals, and others using Pan-Tompkins method."""
    try:
        # Detect R-peaks using NeuroKit2's Pan-Tompkins based method (implemented in ECG process)
        signals, info = nk.ecg_process(signal, sampling_rate=sampling_rate)
        rpeaks = info['ECG_R_Peaks']

        # Check if there are enough R-peaks
        if len(rpeaks) < 2:
            print(f"Not enough R-peaks detected. Found: {len(rpeaks)}")
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ""

        # RR intervals
        rr_intervals = np.diff(rpeaks) / sampling_rate

        if len(rr_intervals) == 0:
            print("No RR intervals calculated.")
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ""

        # BPM Calculation
        bpm = 60 / np.mean(rr_intervals)

        # RR Interval stats (average, min, max, median)
        average_rr = np.mean(rr_intervals)
        min_rr = np.min(rr_intervals)
        max_rr = np.max(rr_intervals)
        median_rr = np.median(rr_intervals)

        # PT Interval (approximate)
        pt_interval = average_rr

        # Local RR differences (sliding window of 10)
        local_rr = [abs(rr_intervals[i] - rr_intervals[i+10]) for i in range(len(rr_intervals)-10)]

        return bpm, average_rr, min_rr, max_rr, pt_interval, median_rr, ','.join(map(str, local_rr))

    except ZeroDivisionError as e:
        print(f"Error calculating heart features: integer division or modulo by zero")
        return None, None, None, None, None, None, "integer division or modulo by zero"

    except Exception as e:
        print(f"Error calculating heart features: {e}")
        return None, None, None, None, None, None, str(e)

def process_single_ecg(file_path, header_path, sampling_rate=500, downsample_factor=10, plot_signal=False, failed_log_file="failed_ecg_files.log"):
    """Process one ECG file and return extracted features."""
    try:
        mat_data = scipy.io.loadmat(file_path)
        ecg_data = mat_data['val']
        
        # Select Lead II (assuming second lead)
        lead_II = ecg_data[1]

        # Preprocess
        lead_II = preprocess_signal(lead_II, original_fs=sampling_rate, downsample_factor=downsample_factor)
        new_sampling_rate = sampling_rate // downsample_factor

        # Extract features
        bpm, avg_rr, min_rr, max_rr, pt_interval, median_rr, local_rr = calculate_heart_features(lead_II, new_sampling_rate)

        # If R-peak detection failed, log the file and return NaN values
        if bpm is None:
            with open(failed_log_file, "a") as log:
                log.write(f"Failed to process {file_path} - Reason: {local_rr}\n")

            header_info = read_header_info(header_path)
            return {
                'PersonID': os.path.splitext(os.path.basename(file_path))[0],
                'Age': header_info['Age'],
                'Sex': header_info['Sex'],
                'Arrhythmia': header_info['Arrhythmia'],
                'BPM': np.nan,
                'Average_RR_Interval': np.nan,
                'Min_RR_Interval': np.nan,
                'Max_RR_Interval': np.nan,
                'PT_Interval': np.nan,
                'RR_Median': np.nan,
                'Local_RR_Intervals': np.nan
            }

        # If successful, proceed with the normal processing
        header_info = read_header_info(header_path)

        if plot_signal:
            plot_ecg(lead_II, new_sampling_rate, title=os.path.basename(file_path))

        return {
            'PersonID': os.path.splitext(os.path.basename(file_path))[0],
            'Age': header_info['Age'],
            'Sex': header_info['Sex'],
            'Arrhythmia': header_info['Arrhythmia'],
            'BPM': bpm,
            'Average_RR_Interval': avg_rr,
            'Min_RR_Interval': min_rr,
            'Max_RR_Interval': max_rr,
            'PT_Interval': pt_interval,
            'RR_Median': median_rr,
            'Local_RR_Intervals': local_rr
        }
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        with open(failed_log_file, "a") as log:
            log.write(f"Error processing {file_path}: {e}\n")
        return None

def process_all_ecg_files(base_path, downsample_factor=10, output_csv="ecg_data_summary.csv", plot_signals=False):
    """Recursively process all ECG+Header files in a directory sequentially."""
    ecg_files = []
    header_files = []

    # Walk through directories
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.mat'):
                ecg_path = os.path.join(root, file)
                header_path = os.path.join(root, file.replace('.mat', '.hea'))
                if os.path.exists(header_path):
                    ecg_files.append(ecg_path)
                    header_files.append(header_path)
                else:
                    print(f"Header missing for {ecg_path}")

    print(f"ECG files found: {len(ecg_files)}")
    print(f"Header files found: {len(header_files)}")

    if not ecg_files:
        print("No ECG files found! Exiting.")
        return pd.DataFrame()

    results = []

    # Add tqdm progress bar here
    for ecg_path, header_path in tqdm(zip(ecg_files, header_files), total=len(ecg_files), desc="Processing ECG Files"):
        result = process_single_ecg(ecg_path, header_path, downsample_factor=downsample_factor, plot_signal=plot_signals)
        if result:
            results.append(result)
        else:
            print("Skipped a file due to processing failure")

    if results:
        df = pd.DataFrame(results)
        df.to_csv(output_csv, index=False)
        print(f"Data saved to {output_csv}")
        return df
    else:
        print("No valid data processed.")
        return pd.DataFrame()

In [ ]:
def main():
    print("Current working directory:", os.getcwd())

    df = process_all_ecg_files(
        base_path="physionet.org",   # your main folder
        downsample_factor=10,
        output_csv="ecg_data_summary1.csv",
        plot_signals=False,
    )
    print(df.head())

if __name__ == "__main__":
    main()

Current working directory: C:\Users\arche\AIProject\ai-ecg-arrhythmia
ECG files found: 45152
Header files found: 45152


Processing ECG Files:   0%|▍                                                                                                                                                | 129/45152 [00:05<30:29, 24.61it/s]C:\Users\arche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\arche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\arche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn

Error calculating heart features: cannot convert float NaN to integer


Processing ECG Files:   0%|▍                                                                                                                                                | 142/45152 [00:05<29:03, 25.82it/s]C:\Users\arche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\arche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\arche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn

Error calculating heart features: cannot convert float NaN to integer


...

Processing ECG Files:  38%|██████████████████████████████████████████████████████                                                                                         | 17059/45152 [11:59<20:15, 23.11it/s]C:\Users\arche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
Processing ECG Files:  38%|██████████████████████████████████████████████████████                                                                                         | 17063/45152 [11:59<18:14, 25.67it/s]

Error calculating heart features: cannot convert float NaN to integer


Processing ECG Files:  38%|██████████████████████████████████████████████████████                                                                                         | 17066/45152 [11:59<18:39, 25.09it/s]C:\Users\arche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
C:\Users\arche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
Processing ECG Files:  38%|██████████████████████████████████████████████████████                                                                                         | 17070/45152 [12:00<17:37, 26.56it/s]

Error calculating heart features: cannot convert float NaN to integer


Processing ECG Files:  38%|██████████████████████████████████████████████████████▉                                                                                        | 17339/45152 [12:12<22:39, 20.46it/s]